In [1]:
# 会在代码同一级的目录下载训练数据，名称oxford-iiit-pet，1.5G
# 不再训练，直接加载训练后的模型参数swinv2_oxford_pet.pth，并测试。

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 加载数据集
train_dataset = datasets.OxfordIIITPet(root='.', split='trainval', download=True, transform=transform)
test_dataset = datasets.OxfordIIITPet(root='.', split='test', download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

# 1. 定义模型结构
# 创建一个与保存时相同的模型
model = timm.create_model('swinv2_base_window8_256', pretrained=False, num_classes=37)

# 2. 加载模型权重
# 加载保存的权重文件
model.load_state_dict(torch.load('swinv2_oxford_pet.pth'))

# 3. 将模型移动到设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 4. 设置模型为评估模式
model.eval()

# 测试模型
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

C:\Users\jaza7\AppData\Local\Temp\ipykernel_12720\1967001681.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('swinv2_oxford_pet.pth'))


Test Accuracy: 88.74%
